## ENGRI 1120 Flux Balance Analysis of the production mRNA Vaccine BNT-162b2

### Introduction

### Lab setup

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/labs/lab-10-flux-balance-analysis`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/labs/lab-10-flux-balance-analysis/Project.toml`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/labs/lab-10-flux-balance-analysis/Manifest.toml`


In [2]:
# load reqd packages and set paths -
using JLD2
using FileIO
using PrettyTables
using DataFrames
using GLPK

# setup paths -
const _ROOT = pwd();
const _PATH_TO_DATA = joinpath(_ROOT, "data");

In [3]:
include("ENGRI-1120-Lab-10-CodeLib.jl");

In [4]:
# load the model file -
model = load(joinpath(_PATH_TO_DATA, "ENGRI-1120-BNT162b2-Model.jld2"))["model"]

Dict{String, Any} with 7 entries:
  "stochiometric_matrix" => [-1.0 0.0 … 0.0 0.0; -1.0 0.0 … 0.0 -1.0; … ; 0.0 0…
  "list_of_reactions"    => ["TX_BNT_162b2_binding", "TX_BNT_162b2_open", "BNT_…
  "reaction_table"       => 5×7 DataFrame…
  "flux_bounds_array"    => [-1000.0 1000.0; 0.0 1000.0; … ; 0.0 1000.0; 0.0 10…
  "mRNA_sequence"        => ['C', 'U', 'C', 'U', 'U', 'A', 'U', 'U', 'U', 'G'  …
  "list_of_species"      => ["G_BNT_162b2", "T7RNAP", "G_BNT_162b2_T7RNAP_close…
  "gene_sequence"        => ['G', 'A', 'G', 'A', 'A', 'T', 'A', 'A', 'A', 'C'  …

In [5]:
# get stuff from the model data structure -
S = model["stochiometric_matrix"]; # fix the spelling in the model file
flux_bounds_array = model["flux_bounds_array"];
list_of_species = model["list_of_species"];
list_of_reactions = model["list_of_reactions"];

In [6]:
# how many species, and reactions do we have?
(ℳ, ℛ) = size(S)

(16, 5)

In [7]:
model["reaction_table"]

Row,id,forward,reverse,reversibility,LB,UB,ec
,String,String,String,Bool,Float64?,Float64?,String?
1,TX_BNT_162b2_binding,G_BNT_162b2+T7RNAP,G_BNT_162b2_T7RNAP_closed,true,-Inf,Inf,missing
2,TX_BNT_162b2_open,G_BNT_162b2_T7RNAP_closed,G_BNT_162b2_T7RNAP_open,false,0.0,Inf,missing
3,BNT_162b2_transcription,G_BNT_162b2_T7RNAP_open+798*M_atp_c+1004*M_utp_c+1060*M_ctp_c+1312*M_gtp_c+4174*M_h2o_c,mRNA_BNT_162b2+G_BNT_162b2+T7RNAP+4174*M_ppi_c,false,0.0,Inf,2.7.7.6
4,mRNA_BNT_162b2_degradation,mRNA_BNT_162b2,798*M_amp_c+1004*M_ump_c+1060*M_cmp_c+1312*M_gmp_c,false,0.0,Inf,missing
5,RNAP_deactivation,T7RNAP,T7RNAP_inactive,false,0.0,Inf,missing


In [8]:
# setup what's comining into the chip -
ṅ₁ = zeros(ℳ,1);
ṅ₂ = zeros(ℳ,1);

# let's suppose we put the DNA in stream 1 -
ṅ₁[1,1] = 10.0;

# in stream 2, we have the PURExpress components -
ṅ₂[2,1] = 10.0;
ṅ₂[5:8,1] .= 100.0;
ṅ₂[9,1] = 10000.0

# build the species bounds array -
species_bounds_array = [-(ṅ₁ .+ ṅ₂) 10000.0*ones(ℳ,1)];

In [9]:
# build a species input table -
species_input_table_data = Array{Any,2}(undef, ℳ,3);

# populate the table -
for i ∈ 1:ℳ
    species_input_table_data[i,1] = list_of_species[i]
    species_input_table_data[i,2] = ṅ₁[i]
    species_input_table_data[i,3] = ṅ₂[i]
end


# show -
pretty_table(species_input_table_data)

┌───────────────────────────┬────────┬─────────┐
│                    Col. 1 │ Col. 2 │  Col. 3 │
├───────────────────────────┼────────┼─────────┤
│               G_BNT_162b2 │   10.0 │     0.0 │
│                    T7RNAP │    0.0 │    10.0 │
│ G_BNT_162b2_T7RNAP_closed │    0.0 │     0.0 │
│   G_BNT_162b2_T7RNAP_open │    0.0 │     0.0 │
│                   M_atp_c │    0.0 │   100.0 │
│                   M_utp_c │    0.0 │   100.0 │
│                   M_ctp_c │    0.0 │   100.0 │
│                   M_gtp_c │    0.0 │   100.0 │
│                   M_h2o_c │    0.0 │ 10000.0 │
│            mRNA_BNT_162b2 │    0.0 │     0.0 │
│                   M_ppi_c │    0.0 │     0.0 │
│                   M_amp_c │    0.0 │     0.0 │
│                   M_ump_c │    0.0 │     0.0 │
│                   M_cmp_c │    0.0 │     0.0 │
│                   M_gmp_c │    0.0 │     0.0 │
│           T7RNAP_inactive │    0.0 │     0.0 │
└───────────────────────────┴────────┴─────────┘


In [10]:
# setup flux boundsa array -
# we get a default from the model generation system, we can update that if we need to -
ϵ̇_bounds = flux_bounds_array;

In [11]:
list_of_reactions

5-element Vector{String}:
 "TX_BNT_162b2_binding"
 "TX_BNT_162b2_open"
 "BNT_162b2_transcription"
 "mRNA_BNT_162b2_degradation"
 "RNAP_deactivation"

In [12]:
list_of_species

16-element Vector{String}:
 "G_BNT_162b2"
 "T7RNAP"
 "G_BNT_162b2_T7RNAP_closed"
 "G_BNT_162b2_T7RNAP_open"
 "M_atp_c"
 "M_utp_c"
 "M_ctp_c"
 "M_gtp_c"
 "M_h2o_c"
 "mRNA_BNT_162b2"
 "M_ppi_c"
 "M_amp_c"
 "M_ump_c"
 "M_cmp_c"
 "M_gmp_c"
 "T7RNAP_inactive"

In [13]:
# setup the objective coefficient array -
obj_vector = zeros(ℛ);
obj_vector[3] = -1;

In [28]:
# compute the optimal flux, and then estimate the output on the chip
result = compute_optimal_extent(S, ϵ̇_bounds, species_bounds_array, obj_vector; θ = 0.25);

# build a system stream table -
ϵ̇ = result.calculated_flux_array;

# compute the output -
ṅ₃ = (ṅ₁ + ṅ₂) + S*ϵ̇;

In [25]:
system_flux_table_data = Array{Any,2}(undef, ℳ, 4);

# populate the table -
for i ∈ 1:ℳ
    system_flux_table_data[i,1] = list_of_species[i]
    system_flux_table_data[i,2] = ṅ₁[i]
    system_flux_table_data[i,3] = ṅ₂[i]
    system_flux_table_data[i,4] = round(ṅ₃[i], digits=3)
end

# header -
state_table_header = (["Species", "ṅ₁,ᵢ (mol/t)", "ṅ₂,ᵢ (mol/t)", "ṅ₃,ᵢ (mol/t)"]);

# show -
pretty_table(system_flux_table_data; header = state_table_header)

┌───────────────────────────┬──────────────┬──────────────┬──────────────┐
│                   Species │ ṅ₁,ᵢ (mol/t) │ ṅ₂,ᵢ (mol/t) │ ṅ₃,ᵢ (mol/t) │
├───────────────────────────┼──────────────┼──────────────┼──────────────┤
│               G_BNT_162b2 │         10.0 │          0.0 │         10.0 │
│                    T7RNAP │          0.0 │         10.0 │         10.0 │
│ G_BNT_162b2_T7RNAP_closed │          0.0 │          0.0 │          0.0 │
│   G_BNT_162b2_T7RNAP_open │          0.0 │          0.0 │          0.0 │
│                   M_atp_c │          0.0 │        100.0 │       39.177 │
│                   M_utp_c │          0.0 │        100.0 │       23.476 │
│                   M_ctp_c │          0.0 │        100.0 │       19.207 │
│                   M_gtp_c │          0.0 │        100.0 │         -0.0 │
│                   M_h2o_c │          0.0 │      10000.0 │      9681.86 │
│            mRNA_BNT_162b2 │          0.0 │          0.0 │        0.057 │
│                   M_ppi